In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def essBoundaryFun(x):
    return 0.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [5]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.setNaturalRHS(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace, ksp)
rhs.setRhsStencil(mesh, K)

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs)

In [7]:
trainingData = dta.StokesData(range(256))
trainingData.readData(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshapeInputImg()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [9]:
steps = int(50)
for s in range(steps):
    print('step = ', s)
    batchSamples_pf = torch.multinomial(torch.ones(trainingData.nSamplesIn), model.batchSizeN_pf)
#     batchSamples_pc = torch.multinomial(torch.ones(trainingData.nSamplesOut), model.batchSizeN_pc)
    model.optLatentDistStep()
    model.pfStep(batchSamples_pf)
#     model.pcStep(batchSamples_pc)
    

step =  0
loss_pf =  tensor(11633338., grad_fn=<SubBackward0>)
step =  1
loss_pf =  tensor(11155332., grad_fn=<SubBackward0>)
step =  2
loss_pf =  tensor(11041564., grad_fn=<SubBackward0>)
step =  3
loss_pf =  tensor(10911280., grad_fn=<SubBackward0>)
step =  4
loss_pf =  tensor(10784490., grad_fn=<SubBackward0>)
step =  5
loss_pf =  tensor(10654291., grad_fn=<SubBackward0>)
step =  6
loss_pf =  tensor(10504382., grad_fn=<SubBackward0>)
step =  7
loss_pf =  tensor(10336110., grad_fn=<SubBackward0>)
step =  8
loss_pf =  tensor(10127802., grad_fn=<SubBackward0>)
step =  9
loss_pf =  tensor(9893294., grad_fn=<SubBackward0>)
step =  10
loss_pf =  tensor(9649530., grad_fn=<SubBackward0>)
step =  11
loss_pf =  tensor(9416518., grad_fn=<SubBackward0>)
step =  12
loss_pf =  tensor(9239682., grad_fn=<SubBackward0>)
step =  13
loss_pf =  tensor(9083782., grad_fn=<SubBackward0>)
step =  14
loss_pf =  tensor(8854896., grad_fn=<SubBackward0>)
step =  15
loss_pf =  tensor(8611730., grad_fn=<SubBackw

In [10]:
model.plotInputReconstruction()
f = plt.gcf()
f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)

Text(0.5, 0.7, 'Untrained, N = 1184')

In [11]:
f.suptitle('N = 256', fontsize=32, y=.7)

Text(0.5, 0.7, 'N = 256')

In [12]:
model.z_mean

tensor([[ 5.1906, -6.0039,  0.7463,  ..., -0.6863,  2.1015,  1.7864],
        [ 1.4645, -4.0327, -1.4917,  ..., -8.2840, -0.4410, -3.1420],
        [-4.9203,  7.5589, -1.0194,  ..., -8.9325, -1.6319,  7.5968],
        ...,
        [ 0.6648,  1.4951,  4.3544,  ...,  3.4628, -3.4711, -2.3891],
        [ 1.3101,  3.6759, -6.0487,  ...,  2.2782,  1.3084,  7.7367],
        [-0.3935,  0.1130, -3.5781,  ...,  4.7696, -0.1121,  7.8719]],
       requires_grad=True)

In [13]:
sample = model.pfNet(model.z_mean[0, :])

In [14]:
sample.shape

torch.Size([65536])

In [15]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [16]:
plt.imshow(torch.reshape(sample, (model.data.imgResolution, model.data.imgResolution)).detach().numpy() > .5,
                     cmap='binary')

In [17]:
plt.figure()
plt.imshow(torch.reshape(sample, (model.data.imgResolution, model.data.imgResolution)).detach().numpy())

In [18]:
plt.figure()
trainingData.plotMicrostruct(0)

In [19]:
f = rom.getAutogradFun

In [22]:
f.forward(torch.ones(16))

AttributeError: 'function' object has no attribute 'forward'

In [12]:
x = [x for x in range(10, 20)]

In [14]:
for i, xx in enumerate(x):
    print('i = ', i)
    print('xx = ', xx)

i =  0
xx =  10
i =  1
xx =  11
i =  2
xx =  12
i =  3
xx =  13
i =  4
xx =  14
i =  5
xx =  15
i =  6
xx =  16
i =  7
xx =  17
i =  8
xx =  18
i =  9
xx =  19


[range(10, 20)]